<a href="https://colab.research.google.com/github/Ksoro7/first_project/blob/master/fcc_book_recommendation_livre_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-09-01 18:08:15--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   158MB/s    in 0.2s    

2025-09-01 18:08:16 (158 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):


  return recommended_books

In [5]:
# add your code here - consider creating a new cell for each section of code
# Fusion des notations et des livres
df = pd.merge(df_ratings, df_books, on="isbn")
# Aperçu des données fusionnées
df.head()
# Filtrer les utilisateurs avec au moins 200 notations
ratings_per_user = df.groupby('user').size()
valid_users = ratings_per_user[ratings_per_user >= 200].index

# Filtrer les livres avec au moins 100 notations
ratings_per_book = df.groupby('title').size()
valid_books = ratings_per_book[ratings_per_book >= 100].index

# Appliquer les filtres
df_filtered = df[
    (df['user'].isin(valid_users)) &
    (df['title'].isin(valid_books))
]

print("Taille avant filtrage :", len(df))
print("Taille après filtrage :", len(df_filtered))

# Création de la matrice user x book
book_matrix = df_filtered.pivot_table(
    index='user',
    columns='title',
    values='rating'
).fillna(0)

# Conversion en matrice creuse
book_sparse = csr_matrix(book_matrix.values)

print("Dimensions matrice (users x livres):", book_matrix.shape)

# Entraînement du modèle KNN basé sur la similarité cosinus
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(book_sparse.T)  # transpose: comparaison entre livres

def get_recommends(book_title):
    if book_title not in book_matrix.columns:
        return f"'{book_title}' n'existe pas dans la base filtrée."

    # Trouver les 6 plus proches voisins (le premier est le livre lui-même)
    distances, indices = model.kneighbors(
        book_matrix.T.loc[[book_title]], n_neighbors=6
    )

    # Construire la liste des recommandations
    recommendations = []
    for i in range(1, 6):  # on ignore le premier car c'est le livre lui-même
        title = book_matrix.T.index[indices[0][i]]
        distance = distances[0][i]
        recommendations.append([title, float(distance)])

    return [book_title, recommendations]
print(get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))"))


Taille avant filtrage : 1031175
Taille après filtrage : 66730
Dimensions matrice (users x livres): (813, 914)
['The Queen of the Damned (Vampire Chronicles (Paperback))', [['The Vampire Lestat (Vampire Chronicles, Book II)', 0.514513373374939], ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.529854416847229], ['Taltos: Lives of the Mayfair Witches', 0.7326513528823853], ['Interview with the Vampire', 0.7364973425865173], ['The Witching Hour (Lives of the Mayfair Witches)', 0.7371460795402527]]]


In [6]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['Blue Diary', 0.7145698070526123], ['The Lovely Bones: A Novel', 0.715856671333313], ['I Know This Much Is True', 0.7496687769889832], ['Icy Sparks', 0.7646523118019104], ['The Weight of Water', 0.7678344249725342]]]
You haven't passed yet. Keep trying!
